# Neural ODE Classifier (Diffsol)

This tutorial mirrors `examples/integration/neural_ode/train.py` but uses a bite-sized configuration
that runs quickly on CPU. We:

1. Map Fashion-MNIST images to a scalar rate parameter.
2. Integrate a 1-D logistic ODE with `DiffsolModule`.
3. Train the classifier end-to-end and visualise the loss curve.
4. Check gradients with the utilities from `diffsol_pytorch.testing`.


In [ ]:
import math
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from diffsol_pytorch import DiffsolModule, device, testing
from helpers import (
    describe_device,
    gpu_section_mode,
    load_fashion_mnist_subset,
    preferred_device,
    save_cached_json,
    seed_everything,
)

In [ ]:
LOGISTIC_CODE = ("
in = [k]
"
    "k { 0.7 }
"
    "u { u = 1.0, }
"
    "F { -k * u, }
")

TIMES = torch.linspace(0.0, 1.0, 41, dtype=torch.float64)
TIMES_LIST = TIMES.tolist()
GRAD_OUT_FINAL = [0.0] * (len(TIMES_LIST) - 1) + [1.0]
MODULE = DiffsolModule(LOGISTIC_CODE)


In [ ]:
class DiffsolLogistic(torch.autograd.Function):
    @staticmethod
    def forward(ctx, rates: torch.Tensor) -> torch.Tensor:
        outputs = []
        for rate in rates.detach().cpu().tolist():
            _, _, flat = MODULE.solve_dense([rate], TIMES_LIST)
            outputs.append(flat[-1])
        u_final = torch.tensor(outputs, dtype=rates.dtype, device=rates.device)
        probs = 1.0 - u_final
        ctx.save_for_backward(rates.detach())
        return probs.clamp(1e-6, 1 - 1e-6)

    @staticmethod
    def backward(ctx, grad_output: torch.Tensor):
        (rates,) = ctx.saved_tensors
        grads = []
        grad_out = np.array(GRAD_OUT_FINAL, dtype=np.float64)
        for rate, upstream in zip(rates.tolist(), grad_output.detach().cpu().tolist()):
            g = testing.reverse_mode_gradients(LOGISTIC_CODE, [rate], TIMES_LIST, grad_out)[0]
            grads.append(-g * upstream)
        grad_tensor = torch.tensor(grads, dtype=grad_output.dtype, device=grad_output.device)
        return grad_tensor


class DiffsolClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(8, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True),
        )
        self.fc = nn.Linear(16 * 7 * 7, 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        z = self.encoder(x)
        rates = torch.nn.functional.softplus(self.fc(z.flatten(1))) + 1e-3
        probs = DiffsolLogistic.apply(rates.squeeze(1))
        return probs


In [ ]:
seed_everything(0)
loader = load_fashion_mnist_subset(samples=128, batch_size=32)
device_target = preferred_device()
print(f"Using device: {describe_device(device_target)}")


In [ ]:
if device_target.type != 'cuda':
    print('CUDA not available; running training on CPU. Enable a GPU runtime to compare performance.')
else:
    print(f'CUDA available on {torch.cuda.get_device_name(0)}')


In [ ]:
model = DiffsolClassifier().to(device_target)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCELoss()
loss_history = []

model.train()
for _ in range(1):
    for images, labels in loader:
        images = images.to(device_target, dtype=torch.float32)
        targets = (labels % 2 == 0).float().to(device_target)
        preds = model(images)
        loss = criterion(preds, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_history.append(loss.item())
loss_history[:5], len(loss_history)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 3))
plt.plot(loss_history, label="BCELoss")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.title("Training curve (1 epoch, 128 samples)")
plt.legend()
plt.show()


In [ ]:
def evaluate(model: nn.Module, loader) -> float:
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device_target, dtype=torch.float32)
            targets = (labels % 2 == 0).to(device_target)
            preds = model(images)
            predicted = (preds > 0.5).long()
            correct += (predicted == targets).sum().item()
            total += targets.numel()
    model.train()
    return correct / max(1, total)

accuracy = evaluate(model, loader)
accuracy


In [ ]:
mode, cached_metrics = gpu_section_mode(
    "Neural ODE GPU benchmark", cache_key="neural_ode_gpu_metrics.json"
)
if mode == "run":
    benchmark_device = preferred_device()
    batch = next(iter(loader))
    images = batch[0].to(benchmark_device, dtype=torch.float32)
    model_eval = model.to(benchmark_device).eval()
    if benchmark_device.type == "cuda":
        torch.cuda.synchronize()
    start = time.perf_counter()
    with torch.inference_mode():
        _ = model_eval(images)
        if benchmark_device.type == "cuda":
            torch.cuda.synchronize()
    latency_ms = (time.perf_counter() - start) * 1_000.0
    metrics = {
        "device": describe_device(benchmark_device),
        "batch_latency_ms": round(latency_ms, 3),
        "batch_size": int(images.shape[0]),
    }
    save_cached_json("neural_ode_gpu_metrics.json", metrics)
    model.to(device_target).train()
elif mode == "cache":
    metrics = cached_metrics
else:
    metrics = {
        "device": "cpu",
        "note": "GPU benchmark skipped; set NB_FORCE_GPU=1 to require an accelerator.",
    }
metrics

In [ ]:
def loss_fn(solution: np.ndarray):
    grad = np.zeros_like(solution)
    grad[0, -1] = 1.0
    return float(solution[0, -1]), grad

results = testing.check_gradients(LOGISTIC_CODE, [0.7], TIMES_LIST, loss_fn)
results["finite_difference"], results["reverse_mode"]
